In [92]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

!wget -O 'artificial_intelligence.json' 'https://kg.diffbot.com/kg/dql_endpoint?type=query&token=your-token&query=type%3AOrganization+industries%3A%22Artificial+Intelligence+Companies%22&size=1000'

In [ ]:
import gdown

url = "https://drive.google.com/uc?id=1owa_NIBnj-Q5uwDyM2xBJeZPipjmkJsH"
output = "artificial_intelligence.json"

gdown.download(url, output, quiet=False)

In [4]:
import json

with open("artificial_intelligence.json") as f:
    output = json.load(f)

<IPython.core.display.Javascript object>

In [89]:
data = output["data"]

<IPython.core.display.Javascript object>

In [90]:
data[0].keys()

dict_keys(['twitterUri', 'nbActiveEmployeeEdges', 'type', 'allNames', 'revenue', 'yearlyRevenues', 'logo', 'id', 'stock', 'nbOrigins', 'sicClassification', 'foundingDate', 'image', 'images', 'wikipediaUri', 'irsEmployerIdentificationNumbers', 'diffbotUri', 'nbIncomingEdges', 'nbEmployeesMin', 'ipo', 'parentCompany', 'angellistUri', 'name', 'motto', 'nbEmployeesMax', 'totalInvestment', 'allOriginHashes', 'linkedInUri', 'naicsClassification', 'nbEmployees', 'githubUri', 'isDissolved', 'importance', 'origin', 'description', 'homepageUri', 'founders', 'ceo', 'investments', 'blogUri', 'descriptors', 'isNonProfit', 'origins', 'isPublic', 'categories', 'crawlTimestamp', 'nbUniqueInvestors', 'facebookUri', 'secCentralIndexKeys', 'summary', 'types', 'boardMembers', 'allUris', 'nbLocations', 'crunchbaseUri', 'industries', 'allDescriptions', 'location', 'locations', 'subsidiaries'])

<IPython.core.display.Javascript object>

In [129]:
data[17]["name"]

'PANASONIC CORP'

<IPython.core.display.Javascript object>

In [95]:
from datetime import datetime

def get_founding_year(company: dict):
    date_str = company.get("foundingDate", {}).get("str")
    if date_str:
        return date_str.split("-")[0][1:]
    return date_str

<IPython.core.display.Javascript object>

In [96]:
names = [company.get("name") for company in data]
revenues = [company.get("revenue", {}).get("value") for company in data]
nbEmployees = [company.get("nbEmployees", None) for company in data]
founding_year = [get_founding_year(company) for company in data]
locations = [
    company.get("location", {}).get("country", {}).get("name") for company in data
]
isPublic = [company.get("isPublic") for company in data]
industries = [company.get("industries") for company in data]

<IPython.core.display.Javascript object>

In [97]:
import pandas as pd

<IPython.core.display.Javascript object>

In [136]:
df = pd.DataFrame(
    {
        "company": [company.get("name") for company in data],
        "revenue": [company.get("revenue", {}).get("value") for company in data],
        "nbEmployees": [company.get("nbEmployees", None) for company in data],
        "founding_year": [get_founding_year(company) for company in data],
        "locations": [
            company.get("location", {}).get("country", {}).get("name")
            for company in data
        ],
        "isPublic": [company.get("isPublic") for company in data],
        "industries": [company.get("industries") for company in data],
    }
)

df = df[df.notnull().all(axis=1)]
df["founding_year"] = df["founding_year"].astype(int)
df["year_from_now"] = df["founding_year"].apply(lambda row: datetime.now().year - row)

<IPython.core.display.Javascript object>

In [137]:
df["isPublic"] = df["isPublic"].map({True: "public", False: "private"})

<IPython.core.display.Javascript object>

In [138]:
def turn_list_into_a_string(industries: list):
    industries = str(industries).replace("[", "").replace("]", "").replace("'", "")
    return industries

<IPython.core.display.Javascript object>

In [139]:
df["industries"] = df["industries"].apply(remove_bracket)

<IPython.core.display.Javascript object>

In [141]:
df.head(5)

,company,revenue,nbEmployees,founding_year,locations,isPublic,industries,year_from_now
0,Google,6.600100e+10,150000,1998,United States of America,public,"Manufacturing Companies, Electronic Products M...",23
1,ABB,3.431200e+10,60000,1988,Switzerland,public,"Software Companies, Artificial Intelligence Co...",33
2,Google,4.900000e+08,35000,1998,United States of America,private,"Software Companies, Artificial Intelligence Co...",23
3,Rockwell Automation Inc.,6.666000e+09,23500,1903,United States of America,public,"Manufacturing Companies, Software Companies, A...",118
4,Keyence,4.958000e+09,7500,1974,Japan,public,"Manufacturing Companies, Electronic Products M...",47


<IPython.core.display.Javascript object>

In [102]:
def human_format(num: float):
    """Turn a number into human-readable format"""
    num = float("{:.3g}".format(num))
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    return "{}{}".format(
        "{:f}".format(num).rstrip("0").rstrip("."), ["", "K", "M", "B", "T"][magnitude]
    )

<IPython.core.display.Javascript object>

In [103]:
def num_to_str(number: float, median: int, median_str: str, column: str):
    if number <= median:
        return f"{column} <= {median_str}"
    else:
        return f"{column} > {median_str}"

<IPython.core.display.Javascript object>

In [104]:
def map_values_based_on_median(column: str, data: pd.DataFrame):
    df = data.copy()
    median = df[column].median()
    median_str = human_format(median)
    df[column] = df[column].apply(
        lambda row: num_to_str(row, median, median_str, column)
    )
    return df

<IPython.core.display.Javascript object>

In [105]:
df = map_values_based_on_median("revenue", df)

<IPython.core.display.Javascript object>

In [106]:
df = map_values_based_on_median("nbEmployees", df)

<IPython.core.display.Javascript object>

In [107]:
df = map_values_based_on_median("year_from_now", df)

<IPython.core.display.Javascript object>

In [110]:
df

,company,revenue,nbEmployees,founding_year,locations,isPublic,industries,year_from_now
0,Google,revenue > 24M,nbEmployees > 350,1998,United States of America,public,"Manufacturing Companies, Electronic Products M...",year_from_now > 19
1,ABB,revenue > 24M,nbEmployees > 350,1988,Switzerland,public,"Software Companies, Artificial Intelligence Co...",year_from_now > 19
2,Google,revenue > 24M,nbEmployees > 350,1998,United States of America,private,"Software Companies, Artificial Intelligence Co...",year_from_now > 19
3,Rockwell Automation Inc.,revenue > 24M,nbEmployees > 350,1903,United States of America,public,"Manufacturing Companies, Software Companies, A...",year_from_now > 19
4,Keyence,revenue > 24M,nbEmployees > 350,1974,Japan,public,"Manufacturing Companies, Electronic Products M...",year_from_now > 19
...,...,...,...,...,...,...,...,...
995,WITHIN,revenue <= 24M,nbEmployees <= 350,2014,United States of America,private,"Software Companies, Artificial Intelligence Co...",year_from_now <= 19
996,Primer,revenue <= 24M,nbEmployees <= 350,2015,United States of America,private,"Software Companies, Artificial Intelligence Co...",year_from_now <= 19
997,Omdena,revenue <= 24M,nbEmployees <= 350,2019,United States of America,private,"Software Companies, Artificial Intelligence Co...",year_from_now <= 19
998,Clerk.io,revenue <= 24M,nbEmployees <= 350,2011,Denmark,private,"Software Companies, Artificial Intelligence Co...",year_from_now <= 19


<IPython.core.display.Javascript object>

In [111]:
df.to_csv("artificial_intelligence.csv", index=False)

<IPython.core.display.Javascript object>

In [70]:
df.explode("industries")["industries"].value_counts()

Software Companies                       86
Artificial Intelligence Companies        86
Robotics Companies                       29
Natural Language Processing Companies    20
Manufacturing Companies                  15
Computer Hardware Companies               8
Computer Vision Companies                 8
Electronic Products Manufacturers         6
Enterprise Software Companies             5
Virtual Reality Companies                 5
Machine Manufacturers                     5
Retailers                                 4
Financial Technology Companies            4
Financial Services Companies              4
Professional Service Companies            4
Software As A Service Companies           4
Motor Vehicle Manufacturers               4
Tool Manufacturers                        4
Home Appliance Manufacturers              2
Coworking Spaces                          2
Engine Manufacturers                      2
Public Administration                     2
Display Technology Companies    

<IPython.core.display.Javascript object>

In [132]:
import gdown

url = "https://drive.google.com/uc?id=1gQ4TEqhC_5nyHz-Uje5SSrvHoHuSXFWc"
output = "artificial_intelligence.csv"

gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1gQ4TEqhC_5nyHz-Uje5SSrvHoHuSXFWc
To: /home/khuyen/Data-science/visualization/artificial_intelligence.csv
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 176k/176k [00:00<00:00, 180kB/s]


'artificial_intelligence.csv'

<IPython.core.display.Javascript object>